# MSc Thesis - Hugo Stam

### Example 04 - Onshore and Offshore Port System Analysis

<img src="Figures/system_layout_alternative_1.2.png" style="width:1000px">
<img src="Figures/system_layout_alternative_2.2.png" style="width:1000px">
<img src="Figures/system_layout_alternative_3.2.png" style="width:1000px">

##### Investment Strategy Simulation - Adaptive terminal planning in the light of an uncertain future

* [**0. Prepare inputs:**](#0.-Prepare-inputs)<br>
   * [**0.1 Generate demand forecast scenario:**](#0.1-Generate-demand-forecast-scenario)<br>
   * [**0.2 Generate vessels:**](#0.2-Generate-vessels)<br>
* [**1. Instatiate system components:**](#1.-Instatiate-system-components)<br>
* [**2. Start simulation:**](#2.-Strart-simulation)<br>
* [**3. Report all elements:**](#3.-Report-all-elements)<br>

#### notes:


In [1]:
# packages for data handling
import numpy as np
import pandas as pd
import statistics as st

# packages related to time, space and id
import datetime, time
import platform
import random
from datetime import timedelta

# you need these dependencies packages related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# packages for figures
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import seaborn as sns
# sns.set(style="ticks")
# sns.set(style="darkgrid", palette = "pastel")
# sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1.5})

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# OpenTISim package
from opentisim import container_objects
from opentisim import container_defaults
from opentisim import container_system_onshore as container_system_onshore
from opentisim import container_system_offshore_barge as container_system_offshore_barge
from opentisim import container_system_offshore_bridge as container_system_offshore_bridge

# OpenCLSim package
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# Additional import to save the initialisation of the simulation
import openclsim.savesim as savesim

# Pretty-print a Python object to a stream
from pprint import pprint

# Jupyter Widgets
import ipywidgets as widgets

# Scroll to specific cell
from jupyter_helpers.utilities import scroll_to_current_cell

# 0. Prepare inputs

In [2]:
# iPython Widgests
annual_demand = widgets.IntSlider(
    value=1000000, min=500000, max=1500000, step=50000,
    description='Demand:', orientation='horizontal',
    readout=True, readout_format='d')

offshore_onshore_distance = widgets.IntSlider(
    value=40, min=20, max=60, step=20,
    description='Distance:', orientation='horizontal',
    readout=True, readout_format='d')

container_ship = widgets.Dropdown(
    options=[('Post Panamax I (6,000 TEU)', 6000), 
             ('New-Panamax (12,500 TEU)', 12500), 
             ('ULCS (21,000 TEU)', 21000)],
    value=12500, 
    description='OGV:')

barge = widgets.Dropdown(
    options=[('Small (200 TEU)', 200), 
             ('Medium (250 TEU)', 250), 
             ('Large (300 TEU)', 300)],
    value=250, 
    description='Barge:')

truck = widgets.Dropdown(
    options=[('Small (2 TEU)', 2), 
             ('Medium (2.5 TEU)', 2.5), 
             ('Large (3 TEU)', 3)],
    value=2, 
    description='Truck:')

life_cycle = widgets.IntSlider(
    value=10, min=5, max=30, step=5,
    description='Life cycle:', orientation='horizontal',
    readout=True, readout_format='d')

print('Input Values')
display(annual_demand, offshore_onshore_distance, container_ship, barge, truck, life_cycle)

Input Values


IntSlider(value=1000000, description='Demand:', max=1500000, min=500000, step=50000)

IntSlider(value=40, description='Distance:', max=60, min=20, step=20)

Dropdown(description='OGV:', index=1, options=(('Post Panamax I (6,000 TEU)', 6000), ('New-Panamax (12,500 TEU…

Dropdown(description='Barge:', index=1, options=(('Small (200 TEU)', 200), ('Medium (250 TEU)', 250), ('Large …

Dropdown(description='Truck:', options=(('Small (2 TEU)', 2), ('Medium (2.5 TEU)', 2.5), ('Large (3 TEU)', 3))…

IntSlider(value=10, description='Life cycle:', max=30, min=5, step=5)

In [3]:
offshore_distance = offshore_onshore_distance.value
demand = annual_demand.value
design_container_ship = container_ship.value
design_barge = barge.value
design_truck = truck.value
lifecycle = life_cycle.value

print('The offshore distance:', offshore_distance, 'km')
print('The annual demand:', demand, 'TEU')
print('The design container ship capacity:', design_container_ship, 'TEU')
print('The design barge capacity:', design_barge, 'TEU')
print('The design truck capacity:', design_truck, 'TEU')
print('The design life cycle:', lifecycle, 'years')

avg_parcel = design_container_ship / 8

startyear = 2020

The offshore distance: 40 km
The annual demand: 1000000 TEU
The design container ship capacity: 12500 TEU
The design barge capacity: 250 TEU
The design truck capacity: 2 TEU
The design life cycle: 10 years


## 0.1 Generate demand forecast scenario

In [4]:
# total_ocean_transport = 1000000  # TEU
# years = list(range(startyear, startyear+lifecycle))
# demand_list = []

# for year in years:
#     if year < 2025:
#         demand.append(total_ocean_transport)
#     elif year < 2030:
#         demand.append(total_ocean_transport * 1.5)
#     else:
#         demand.append(total_ocean_transport * 2.0)

# plt.plot(years, demand, 'o')
# plt.ylabel('Demand [TEU]')

In [5]:
years = list(range(startyear, startyear + lifecycle))
        
scenario_data={'year': years, 'volume': demand}

# instantiate Commodity objects, the inputs for the Commodity class
if design_container_ship == 6000:
    container_defaults.container_data['post_panamax_I_perc'] = 100
    container_defaults.container_data['new_panamax_perc'] = 0
    container_defaults.container_data['ULCS_perc'] = 0

if design_container_ship == 12500:
    container_defaults.container_data['post_panamax_I_perc'] = 0
    container_defaults.container_data['new_panamax_perc'] = 100
    container_defaults.container_data['ULCS_perc'] = 0
    
if design_container_ship == 21000:
    container_defaults.container_data['post_panamax_I_perc'] = 0
    container_defaults.container_data['new_panamax_perc'] = 0
    container_defaults.container_data['ULCS_perc'] = 100
    
print(container_defaults.container_data)

# ladens
container = container_objects.Commodity(**container_defaults.container_data)
container.scenario_data = pd.DataFrame(data=scenario_data)
# print(container.scenario_data)

# combine
demand = [container]

{'name': 'Laden', 'handling_fee': 150, 'fully_cellular_perc': 0, 'panamax_perc': 0, 'panamax_max_perc': 0, 'post_panamax_I_perc': 0, 'post_panamax_II_perc': 0, 'new_panamax_perc': 100, 'VLCS_perc': 0, 'ULCS_perc': 0}


## 0.2 Generate vessels

In [6]:
# instantiate vessels
fully_cellular = container_objects.Vessel(**container_defaults.fully_cellular_data)
panamax = container_objects.Vessel(**container_defaults.panamax_data)
panamax_max = container_objects.Vessel(**container_defaults.panamax_max_data)
post_panamax_I = container_objects.Vessel(**container_defaults.post_panamax_I_data)
post_panamax_II = container_objects.Vessel(**container_defaults.post_panamax_II_data)
new_panamax = container_objects.Vessel(**container_defaults.new_panamax_data)
VLCS = container_objects.Vessel(**container_defaults.VLCS_data)
ULCS = container_objects.Vessel(**container_defaults.ULCS_data)

vessels = [fully_cellular, panamax, panamax_max, post_panamax_I, post_panamax_II, new_panamax, VLCS, ULCS] 

# 1. Instatiate system components

## 1.1 Terminal system

Specify the variables

In [7]:
container_system_onshore.PortSystem = container_system_onshore.System(
                                      startyear = startyear,
                                      lifecycle = lifecycle,
                                      stack_equipment='rs',
                                      laden_stack='rs',
                                      crane_type_defaults=container_defaults.sts_crane_data,
                                      allowable_berth_occupancy = 0.50,
                                      transhipment_ratio = 0.0,
                                      elements = demand + vessels,
                                      laden_perc = 0.80, 
                                      reefer_perc = 0.10, 
                                      empty_perc = 0.05, 
                                      oog_perc = 0.05,
                                      operational_hours = 8640,
                                      debug=True)

container_system_offshore_barge.PortSystem = container_system_offshore_barge.System(
                                             startyear = startyear,
                                             lifecycle = lifecycle,
                                             stack_equipment='rs',
                                             laden_stack='rs',
                                             crane_type_defaults=container_defaults.sts_crane_data,
                                             offshore_distance = offshore_distance,
                                             allowable_berth_occupancy = 0.50,
                                             transhipment_ratio = 0.0,
                                             elements = demand + vessels,
                                             laden_perc = 0.80, 
                                             reefer_perc = 0.10, 
                                             empty_perc = 0.05, 
                                             oog_perc = 0.05,
                                             operational_hours = 8640,
                                             debug=True)

container_system_offshore_bridge.PortSystem = container_system_offshore_bridge.System(
                                             startyear = startyear,
                                             lifecycle = lifecycle,
                                             stack_equipment='rs',
                                             laden_stack='rs',
                                             crane_type_defaults=container_defaults.sts_crane_data,
                                             offshore_distance = offshore_distance,
                                             allowable_berth_occupancy = 0.50,
                                             transhipment_ratio = 0.0,
                                             elements = demand + vessels,
                                             laden_perc = 0.80, 
                                             reefer_perc = 0.10, 
                                             empty_perc = 0.05, 
                                             oog_perc = 0.05,
                                             operational_hours = 8640, 
                                             debug=True)

Onshore = container_system_onshore.PortSystem
OffshoreBarge = container_system_offshore_barge.PortSystem
OffshoreBridge = container_system_offshore_bridge.PortSystem

# 2. Start simulation

This method automatically generates investment decisions, parametrically derived from demand trends and a number of investment triggers.

Apply frame of reference style decisions while stepping through each year of the terminal lifecycle and check if investment is needed (in light of strategic objective, operational objective, QSC, decision recipe, intervention method):
    1. for each year evaluate the demand of each commodity (see 0.1 Demand forecast scenario)
    2. for each year evaluate the various investment decisions
    3. for each year calculate the energy costs (requires insight in realized demands)
    4. for each year calculate the demurrage costs (requires insight in realized demands)
    5. for each year calculate terminal revenues
    6. collect all cash flows (capex, opex, revenues)
    7. calculate PV's and aggregate to NPV

In [8]:
Onshore.simulate()
OffshoreBarge.simulate()
OffshoreBridge.simulate()


Onshore Terminal

Below, the various investment decisions are evaluated for the year 2020.

Simulate year: 2020
  Total vessel calls: 640
  Fully cellular calls: 0
  Panamax calls: 0
  Panamax max calls: 0
  Post Panamax I calls: 0
  Post Panamax II calls: 0
  New Panamax calls: 640
  VLCS calls: 0
  ULCS calls: 0
  Total cargo volume: 1000000
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned

  Start analysis:
     Berth occupancy planned (@ start of year): inf
     Berth occupancy online  (@ start of year): inf
     Crane occupancy planned (@ start of year): inf
     Crane occupancy online  (@ start of year): inf

  *** add Berth to elements
     Berth occupancy planned (after adding Berth): inf
     Berth occupancy online  (after adding Berth): inf

  *** add Quay to elements
   Year  Terminal Capex  Maintenance  Insurance
0  2020      24334677.0          0.0        0.0
1  2021      162231

c:\checkouts\opentisim\opentisim\container_system_onshore.py:1733: FutureWarning:

`item` has been deprecated and will be removed in a future version



AttributeError: 'Channel' object has no attribute 'terminal_capex'

## 3. Report all elements

In [ ]:
if True: 
    for element in Onshore.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__) # This is the dictionary containing the module's symbol table.
        
if True: 
    for element in OffshoreBarge.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__) # This is the dictionary containing the module's symbol table.
        
if True: 
    for element in OffshoreBridge.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__) # This is the dictionary containing the module's symbol table.

In [ ]:
# scroll_to_current_cell(preserve=True)

In [ ]:
# Onshore.terminal_elements_plot()
# OffshoreBarge.terminal_elements_plot()
# OffshoreBridge.terminal_elements_plot()

In [ ]:
# Onshore.terminal_capacity_plot()
# OffshoreBarge.terminal_capacity_plot()
# OffshoreBridge.terminal_capacity_plot()

In [ ]:
# Onshore.terminal_land_use_plot()
# OffshoreBarge.terminal_land_use_plot()
# OffshoreBridge.terminal_land_use_plot()

In [ ]:
# Onshore.storage_area_plot()
# OffshoreBarge.storage_area_plot()
# OffshoreBridge.storage_area_plot()

#### Add cash flow information for each of the terminal elements.

In [ ]:
(Onshore.terminal_capex, Onshore.terminal_opex, 
 Onshore.cash_flows, Onshore.cash_flows_df, 
 Onshore.cash_flows_WACC_real_df, Onshore.NPV) = Onshore.net_present_value()

(OffshoreBarge.terminal_capex, OffshoreBarge.terminal_opex, 
 OffshoreBarge.capital_dredging, OffshoreBarge.maintenance_dredging, 
 OffshoreBarge.barge_capex, OffshoreBarge.barge_opex, 
 OffshoreBarge.cash_flows, OffshoreBarge.cash_flows_df, 
 OffshoreBarge.cash_flows_WACC_real_df, OffshoreBarge.NPV) = OffshoreBarge.net_present_value()

(OffshoreBridge.terminal_capex, OffshoreBridge.terminal_opex, 
 OffshoreBridge.bridge_construction, OffshoreBridge.bridge_maintenance_dredging, 
 OffshoreBridge.truck_capex, OffshoreBridge.truck_opex, 
 OffshoreBridge.cash_flows, OffshoreBridge.cash_flows_df, 
 OffshoreBridge.cash_flows_WACC_real_df, OffshoreBridge.NPV) = OffshoreBridge.net_present_value()

In [ ]:
# Onshore.terminal_opex_plot(Onshore.cash_flows_df)
# OffshoreBarge.terminal_opex_plot(OffshoreBarge.cash_flows_df)
# OffshoreBridge.terminal_opex_plot(OffshoreBridge.cash_flows_df)

In [ ]:
# Onshore.total_opex_plot(Onshore.cash_flows_df)
# OffshoreBarge.total_opex_plot(OffshoreBarge.cash_flows_df)
# OffshoreBridge.total_opex_plot(OffshoreBridge.cash_flows_df)

In [ ]:
# print('\033[1mCash Flow Plot\033[0m')
# Onshore.cashflow_plot(Onshore.cash_flows_df)
# OffshoreBarge.cashflow_plot(OffshoreBarge.cash_flows_df)
# OffshoreBridge.cashflow_plot(OffshoreBridge.cash_flows_df)

In [ ]:
terminal_opex_list = ['Maintenance','Insurance','Energy','Labour', 'Fuel']

print('\033[1mFinancial Indication - Onshore Port\033[0m')
print('')
print('*** NPV: $ {}'.format(f'{round(Onshore.NPV,-6):,}'))
Onshore.cash_flows_WACC_real_df.style.hide_columns(terminal_opex_list).hide_index()

In [ ]:
print('\033[1mFinancial Indication - Offshore Port (Barge)\033[0m')
print('')
print('*** NPV: $ {}'.format(f'{round(OffshoreBarge.NPV,-6):,}'))
OffshoreBarge.cash_flows_WACC_real_df.style.hide_columns(terminal_opex_list).hide_index()

In [ ]:
print('\033[1mFinancial Indication - Offshore Port (Bridge)\033[0m')
print('')
print('*** NPV: $ {}'.format(f'{round(OffshoreBridge.NPV,-6):,}'))
OffshoreBridge.cash_flows_WACC_real_df.style.hide_columns(terminal_opex_list).hide_index()

In [ ]:
print('\033[1m Cash Flow Plot - Weighted average cost of capital \033[0m')
Onshore.cashflow_plot(Onshore.cash_flows_WACC_real_df)
OffshoreBarge.cashflow_plot(OffshoreBarge.cash_flows_WACC_real_df)
OffshoreBridge.cashflow_plot(OffshoreBridge.cash_flows_WACC_real_df)

In [ ]:
NPVs = [round(Onshore.NPV,-6), round(OffshoreBarge.NPV,-6), round(OffshoreBridge.NPV,-6)]
print(NPVs)

NPV 40 km, New-Panamax

[-519000000.0, -1206000000.0, -1234000000.0]

In [ ]:
if offshore_distance == 20:
    print('The NPV of alt. 1 for 20 km offshore', NPVs[0])
    print('The NPV of alt. 2 for 20 km offshore', NPVs[1])
    print('The NPV of alt. 3 for 20 km offshore', NPVs[2])
else:
    pass

if offshore_distance == 40:
    print('The NPV of alt. 1 for 40 km offshore', NPVs[0])
    print('The NPV of alt. 2 for 40 km offshore', NPVs[1])
    print('The NPV of alt. 3 for 40 km offshore', NPVs[2])
else:
    pass

if offshore_distance == 60:
    print('The NPV of alt. 1 for 60 km offshore', NPVs[0])
    print('The NPV of alt. 2 for 60 km offshore', NPVs[1])
    print('The NPV of alt. 3 for 60 km offshore', NPVs[2])
else:
    pass